In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 21829381
paper_name = 'fei_yang_2011' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_csv('raw_data/hits.txt', sep='\t')

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 11 x 3


In [10]:
original_data.head()

,Unnamed: 0,SC,YPD
0,YLR404W,66.1,22.3
1,YGR157W,60.9,0.5
2,YJR073C,89.3,0.7
3,YDR123C,97.0,0.3
4,YOL108C,96.3,0.5


In [11]:
original_data['orf'] = original_data['Unnamed: 0'].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

            Unnamed: 0   SC  YPD orf
index_input                         
10                  WT  0.7  0.3  WT


In [15]:
original_data['data'] = original_data['SC']

In [16]:
original_data.set_index('orf', inplace=True)

In [17]:
original_data = original_data[['data']].copy()

In [18]:
original_data = original_data.groupby(original_data.index).mean()

In [20]:
original_data = original_data / original_data.loc['WT']

In [23]:
original_data.drop(index='WT', inplace=True)

In [24]:
original_data.shape

(10, 1)

# Prepare the final dataset

In [26]:
data = original_data.copy()

In [27]:
dataset_ids = [124]
datasets = datasets.reindex(index=dataset_ids)

In [28]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [29]:
data.head()

dataset_id,124
data_type,value
orf,
YBR029C,66.571429
YBR147W,101.428571
YDR123C,138.571429
YGL019W,46.857143
YGR157W,87.000000


## Subset to the genes currently in SGD

In [30]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [31]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,124
,data_type,value
gene_id,orf,
225,YBR029C,66.571429
341,YBR147W,101.428571
2250,YDR123C,138.571429
2701,YGL019W,46.857143
3102,YGR157W,87.000000


# Normalize

In [32]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [33]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [34]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id              124           
data_type             value     valuez
gene_id orf                           
225     YBR029C   66.571429  14.319947
341     YBR147W  101.428571  21.817945
2250    YDR123C  138.571429  29.807615
2701    YGL019W   46.857143  10.079276
3102    YGR157W   87.000000  18.714265

# Print out

In [35]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [36]:
from IO.save_data_to_db3 import *

In [37]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 21829381...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00, 10.94it/s]

Updating the data_modified_on field...
